In [74]:
import json

import advertools as adv
from nltk import word_tokenize

from nltk.corpus import words
from nltk.metrics.distance import edit_distance,jaccard_distance
from nltk.util import ngrams
import pandas as pd
import numpy as np
import enchant
import string 

from tqdm import tqdm

In [75]:
import nltk 
nltk.download('words')
d = enchant.Dict("en_UK")

[nltk_data] Downloading package words to /home/elubrini/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [76]:
with open('./vector.txt','r') as f:
    correct_spellings = f.read().split("\n")

with open('../../data/sample_doc.txt', 'r') as f:
    sample = f.read()

In [77]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [78]:
spellings_series = flatten([gram for spelling in correct_spellings for gram in adv.word_tokenize(spelling)])
spellings_series = pd.Series(spellings_series)
spellings_series

0               chermes pruni
1               pruni scopoli
2                scopoli 1763
3                psylla pruni
4               pruni scopoli
5                scopoli 1763
6            cacopsylla pruni
7               pruni scopoli
8                scopoli 1763
9     cacopsylla thamnopsylla
10         thamnopsylla pruni
11              pruni scopoli
12               scopoli 1763
13               psylla pruni
14              pruni scopoli
15               scopoli 1763
16           cacopsylla pruni
17              pruni scopoli
18               scopoli 1763
19           use(s chresonymy
20           cacopsylla pruni
21              pruni scopoli
22           cacopsylla pruni
23              pruni scopoli
24               scopoli 1763
25              chermes pruni
26              pruni scopoli
27               scopoli 1763
28           cacopsylla pruni
29              pruni scopoli
30               scopoli 1763
31    cacopsylla thamnopsylla
32         thamnopsylla pruni
33        

In [79]:

def jaccard(entry, gram_number, spellings_series):
    spellings = spellings_series[spellings_series.str.startswith(entry[0])] 
    distances = ((jaccard_distance(set(ngrams(entry, gram_number)),
                                    set(ngrams(word, gram_number))), word)
                    for word in spellings)
    try:
        closest = min(distances)
        outcome = (closest[1])
    except ValueError:
        outcome = entry    
    return outcome

In [80]:
def correct(entry, gram_number, spellings_series):
    no_punct_entry = (entry+'.').translate(str.maketrans('', '', string.punctuation))
    if not no_punct_entry:
        return entry
    if d.check(entry) or d.check(no_punct_entry):
        return entry
    else:
        return jaccard(entry, gram_number, spellings_series)
        
def correct_text(text, gram_number, spellings_series, df_out=False, save=False):
    entries = word_tokenize(text)
    corrections = []
    for entry in tqdm(entries):
        correction = correct(entry, gram_number, spellings_series)
        corrections.append(correction)
    if df_out:
        dic = dict(entries=entries,corrections=corrections)
        df = pd.DataFrame(dic)
        df['diff'] = np.where( df['entries'] == df['corrections'] , 1, 0)
        df[df['diff'] == 0]
        return df
    else:
        return corrections

In [81]:
def correct_text_ngrams(text, gram_number, spellings_series, df_out=False, save=False):
    corrections = []
    entries = []
    for spelling in spellings_series:
        print(spelling)
        token_ngram = len(list(word_tokenize(spelling)))
        #print(token_ngram)
        new_entries = adv.word_tokenize(text, token_ngram)[0]
        entries+=new_entries
        for entry in tqdm(new_entries):
            correction = correct(entry, gram_number, spellings_series)
            corrections.append(correction)
    if df_out:
        #print(len(entries), len(corrections))
        dic = dict(entries=entries,corrections=corrections)
        df = pd.DataFrame(dic)
        df['diff'] = np.where( df['entries'] == df['corrections'] , 1, 0)
        df[df['diff'] == 0]
        return df
    else:
        return corrections

In [82]:
sample = " Prawdopodobnie przy uwzgiednieniu wszystkich palearktyeznyeh gatunkéw z rodzaju Cacopsylla przy tworzeniu gru"

In [83]:
correction = correct_text_ngrams(sample, 3, spellings_series, df_out=True)
correction.head()

chermes pruni
['prawdopodobnie przy', 'przy uwzgiednieniu', 'uwzgiednieniu wszystkich', 'wszystkich palearktyeznyeh', 'palearktyeznyeh gatunkéw', 'gatunkéw z', 'z rodzaju', 'rodzaju cacopsylla', 'cacopsylla przy', 'przy tworzeniu', 'tworzeniu gru']


100%|██████████| 11/11 [00:00<00:00, 416.69it/s]


pruni scopoli
['prawdopodobnie przy', 'przy uwzgiednieniu', 'uwzgiednieniu wszystkich', 'wszystkich palearktyeznyeh', 'palearktyeznyeh gatunkéw', 'gatunkéw z', 'z rodzaju', 'rodzaju cacopsylla', 'cacopsylla przy', 'przy tworzeniu', 'tworzeniu gru']


100%|██████████| 11/11 [00:00<00:00, 447.88it/s]


scopoli 1763
['prawdopodobnie przy', 'przy uwzgiednieniu', 'uwzgiednieniu wszystkich', 'wszystkich palearktyeznyeh', 'palearktyeznyeh gatunkéw', 'gatunkéw z', 'z rodzaju', 'rodzaju cacopsylla', 'cacopsylla przy', 'przy tworzeniu', 'tworzeniu gru']


...

pruni scopoli
['prawdopodobnie przy', 'przy uwzgiednieniu', 'uwzgiednieniu wszystkich', 'wszystkich palearktyeznyeh', 'palearktyeznyeh gatunkéw', 'gatunkéw z', 'z rodzaju', 'rodzaju cacopsylla', 'cacopsylla przy', 'przy tworzeniu', 'tworzeniu gru']


100%|██████████| 11/11 [00:00<00:00, 309.21it/s]


scopoli 1763
['prawdopodobnie przy', 'przy uwzgiednieniu', 'uwzgiednieniu wszystkich', 'wszystkich palearktyeznyeh', 'palearktyeznyeh gatunkéw', 'gatunkéw z', 'z rodzaju', 'rodzaju cacopsylla', 'cacopsylla przy', 'przy tworzeniu', 'tworzeniu gru']


100%|██████████| 11/11 [00:00<00:00, 261.65it/s]


,entries,corrections,diff
0,prawdopodobnie przy,psylla pruni,0
1,przy uwzgiednieniu,pruni scopoli,0
2,uwzgiednieniu wszystkich,use(s chresonymy,0
3,wszystkich palearktyeznyeh,wszystkich palearktyeznyeh,1
4,palearktyeznyeh gatunkéw,pruni scopoli,0


In [84]:
correction[correction['diff']==0][['entries','corrections']]

,entries,corrections
0,prawdopodobnie przy,psylla pruni
1,przy uwzgiednieniu,pruni scopoli
2,uwzgiednieniu wszystkich,use(s chresonymy
4,palearktyeznyeh gatunkéw,pruni scopoli
8,cacopsylla przy,cacopsylla pruni
...,...,...
374,uwzgiednieniu wszystkich,use(s chresonymy
376,palearktyeznyeh gatunkéw,pruni scopoli
380,cacopsylla przy,cacopsylla pruni
381,przy tworzeniu,pruni scopoli


## Save

In [85]:
filename = 'sample_correction.json'
correction.to_json(filename)

## Load

In [86]:
df = pd.read_json(filename)
df.head()

,entries,corrections,diff
0,prawdopodobnie przy,psylla pruni,0
1,przy uwzgiednieniu,pruni scopoli,0
2,uwzgiednieniu wszystkich,use(s chresonymy,0
3,wszystkich palearktyeznyeh,wszystkich palearktyeznyeh,1
4,palearktyeznyeh gatunkéw,pruni scopoli,0
